In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-22 11:07:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.08s   

2024-08-22 11:07:58 (13.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [3]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


Google has a sub-word tokenizer called [Setence Piece](), OpenAI has [tiktoken](). 


In [6]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) 

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# Split the dataset
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
# Block size specifies the max length of consecutive tokens taken from the dataset for training
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


Note: For each block we take inputs of all lengths and ask the model to predict during training, this will help the transformer to get used to seeing the context, so that at inference time we can ask it to predict the next token, and concatenate it to the current output and repeat.

Next we generalize to make a batch of blocks of dimension X=[batch_size, block_size], y=[batch_size, block_size], such as `X[i, :]` is the ith block and for input `X[i, :j+1]` the target is `Y[i, j]`.

In [10]:
torch.manual_seed(42)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
targets:
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
----
when input is [57] the target: 1
when input is [57, 1] the target: 46
when input is [57, 1, 46] the target: 47
when input is [57, 1, 46, 47] the target: 57
when input is [57, 1, 46, 47, 57] the target: 1
when input is [57, 1, 46, 47, 57, 1] the target: 50
when input is [57, 1, 46, 47, 57, 1, 50] the target: 53
when input is [57, 1, 46, 47, 57, 1, 50, 53] the target: 60
when input is [1] the target: 58
when input is [1, 58] the target: 46
when input is [1, 58, 46] the target: 43
when input is [1, 58, 46, 43] the target: 56
when input is [1, 58, 46, 43, 56] the target: 43
when input is [1, 58, 46,

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None): # both idx and targets of shape [B, T, C]
        logits = self.token_embedding_table(idx) # idx: [B, T] -> [B, T, C] because each element is mapped to a length C embedding
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # cross ent in pytorch takes inputs with channel at the second dimension

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # sample rather than take max proba, interesting!
            idx = torch.cat((idx, idx_next), dim=1) # extent in the time dimension (dim 1)
        return idx
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits, loss)

tensor([[ 0.4224, -0.8596,  0.2910,  ...,  0.2380, -0.1270,  0.0534],
        [ 0.8564,  2.2181,  0.5232,  ...,  0.4114,  1.9312,  1.0119],
        [ 0.4851,  0.0060,  1.0007,  ..., -0.0776, -0.5015, -2.2270],
        ...,
        [-0.1726, -0.6626, -0.5495,  ..., -0.3630,  0.4550,  0.7595],
        [ 1.0923, -1.3024,  1.3521,  ..., -0.2658, -1.4075, -0.7460],
        [ 0.4851,  0.0060,  1.0007,  ..., -0.0776, -0.5015, -2.2270]],
       grad_fn=<ViewBackward0>) tensor(4.8865, grad_fn=<NllLossBackward0>)


Note: the model as defined is indeed bigram model because it models the next token solely based on the last token via:
```
logits = self.token_embedding_table(idx)
logits = logits[:, -1, :]
probs = F.softmax(logits, dim=-1)
```
That is, given a sequence of token, only the embedding of the last one is directly used to compute the probability for generating the next token. There's no further association beyond that between each token (character) and its predecessor.

In [12]:
# use the initialized model to generate from an initial token (n_batch=1, n_token=1) 100 consecutive tokens
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))



o$,q&IWqW&xtCjaB?ij&bYRGkF?b; f ,CbwhtERCIfuWr,DzJERjhLlVaF&EjffPHDFcNoGIG'&$qXisWTkJPw
 ,b Xgx?D3sj


Let's try training the model and see if the result would be better

In [13]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5233194828033447


In [15]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


QUDUThe chas.
F lisen tabr:
LI mus nk,
A: al l ayo cenghe's therinvar,
TEsen ithawaneit at islinerainy atsomo clour pad d wikn h,
HYy my Tholes:
it GBy ke m vilou xthazinderand llo chee lond Cld this lisesule wars, tirofof wnofan
Rou cthe p.

By hat celis ire m, aksthethe aur withAR wotoot.
Toy:me, of Ithed; bo r:
DWAy celowinoourne,
WIDYoukesu t I:f fowhilong bert irw:
I m;
ADWhit hor hy t I nd, billexve, war t, s
When re llyong thm ithinde!
Whem mire ow
MIAPet mad, trd br hay
ANG w t we illlai


This is still pretty garbage, less garbage I'd say than using 100 epochs, but very far from imiting Shakespeare's works!

Note: To move beyond the bigram, we want tokens to "talk" to each other, namely, have information flow from earlier to later tokens in the sequence when predicting the next token. This introduces the self-attention mechanism.

The simplest interaction would be at each position, take the average of all token embeddings before and use that as the context for the current token position, but this is extremely noisy.

In [16]:
torch.manual_seed(42)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [19]:
# do the above with matrix multiplication with normalized tril matrix
wei = torch.tril(torch.ones(T, T))
wei = wei/wei.sum(1, keepdims=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [20]:
xbow2 = wei @ x 
torch.allclose(xbow, xbow2)

True

In [23]:
# a third way uses softmax, noting that exp(-inf) = 0 and exp(0) = 1
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
# token cannot aggregate with those from the future, interaction strength = -inf
wei = wei.masked_fill(tril==0, float('-inf')) 
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

For the actual attention mechanism, we will not start from a uniform weight matrix. Rather, for each token we produce a key and query, the key describes what information the token contains and the query represents what information the token is seeking when aggregating from tokens before it. These two will result from a simple linear transformation from the token embeddings. For each token, the weight given to tokens before results from using queries to search for keys implemented by a dot product. Thus, if the query of the token is similar to the key of some token before it, that will result in a high weight for that earlier token position.

In [25]:
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1)

The final modification is that instead of aggregating the token embeddings, we also linear transform these embeddings into "values". In this way, we consider the token embedding to be private information to that token, whereas the value to be public information that are shared when computing the aggregate token at other positions.

In [28]:
value = nn.Linear(C, head_size, bias=False)
v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

Notes:
* Attention mechanism is a communication mechanism that can be applied to any directed graphs, and for language modeling only a very special structure is used. In encoder architectures, all nodes connect to one another; in decoder architectures we use a lower triangular mask so only earlier nodes connect to later nodes.
* There's no sense of space for the tokens compared to the convolutional mechanism, and that must be encoded use some spatial encoding.
* Computation is independent in each element along the batch dimension.
* In general cross attention is used when we want to pull information from an additional set of source nodes, such that we can use different set of nodes to compute keys and queries vs. values.

Next for multihead attention, we make several heads that take the same tokens but use different key, query, and value transformations, and the output tokens from each head are concatenated and projected down to the original embedding dimension.

The next step is to add a per-token feedforward network after the attention. If the attention allows the token to talk to each other and transform the embeddings collectively, the feedforward layer let each token "reflect" on what they have learned.

We also interleave several blocks of self-attention and feedforward layers to further improve the model performance. To overcome optimization issues with deep neural nets, we use tricks like residual connections,layer normalization and dropouts.

Layer norm: compared to batch norm, we normalize each dimension of the feature such that after normalization the feature components are about unit gaussian distributed (zero mean, unit variance) for each example in the batch.

### Fun project: let the transformer learn to add (in decimal)

In [95]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

block_size = 9
chars = [str(i) for i in range(10)] + ['+', '=', ' ']
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
itos[-1] = '*'
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


class AdditionProblems(Dataset):
    def __init__(self, size=1000):
        self.size = size
        np.random.seed(42)

    def __len__(self):
        return self.size

    def __getitem__(self, index):
        a = np.random.randint(100)
        b = np.random.randint(100)
        prob = f'{a:2}+{b:2}={str(a+b)[::-1]:4}'
        mask = None
        return encode(prob)

In [73]:
encode('123')

[1, 2, 3]

In [74]:
add_probs = AdditionProblems(1000)
add_probs.__getitem__(2)

[5, 1, 10, 9, 2, 11, 3, 4, 1, 12]

In [61]:
dataloader = DataLoader(add_probs, batch_size=32, shuffle=True)

for probs in dataloader:
    print(len(probs), probs)

32 ['60+20=08 ', '82+86=861', '74+74=841', '87+99=681', '23+ 2=52 ', '21+52=37 ', ' 1+87=88 ', '29+37=66 ', ' 1+63=46 ', '59+20=97 ', '32+75=701', '57+21=87 ', '88+48=631', '90+58=841', '41+91=231', '59+79=831', '14+61=57 ', '61+46=701', '61+50=111', '54+63=711', ' 2+50=25 ', ' 6+20=62 ', '72+38=011', '17+ 3=02 ', '88+59=741', '13+ 8=12 ', '89+52=141', ' 1+83=48 ', '91+59=051', '70+43=311', ' 7+46=35 ', '34+77=111']
32 ['80+35=511', '49+ 3=25 ', ' 1+ 5=6  ', '53+ 3=65 ', '53+92=541', '62+17=97 ', '89+43=231', '33+73=601', '61+99=061', '13+94=701', '47+14=16 ', '71+77=841', '86+61=741', '39+84=321', '79+81=061', '52+23=57 ', '25+88=311', '59+40=99 ', '28+14=24 ', '44+64=801', '88+70=851', ' 8+87=59 ', ' 0+ 7=7  ', '87+62=941', '10+80=09 ', ' 7+34=14 ', '34+32=66 ', ' 4+40=44 ', '27+ 6=33 ', '72+71=341', '11+33=44 ', '32+47=97 ']
32 ['22+61=38 ', '87+36=321', '98+43=141', '85+90=571', '34+64=89 ', '98+46=441', '77+ 2=97 ', ' 0+ 4=4  ', '89+13=201', '26+ 8=43 ', '78+14=29 ', '89+41=031', 

Or just like in our original code, we can bypass using torch's dataset and dataloader framework.

In [97]:
batch_size = 32
block_size = 9

def mask(encoded):
    encoded[:5] = -1
    return encoded

    
def get_batch():
    problems = []
    for _ in range(batch_size):
        a = np.random.randint(100)
        b = np.random.randint(100)
        prob = f'{a:2}+{b:2}={str(a+b)[::-1]:4}'
        problems.append(torch.tensor(encode(prob)))
    x = torch.stack([prob[:block_size] for prob in problems])
    y = torch.stack([mask(prob[1:block_size+1]) for prob in problems])
    return x, y

In [98]:
x, y = get_batch()
print(x.shape, y.shape)

torch.Size([32, 9]) torch.Size([32, 9])


In [99]:
x[0]

tensor([ 9,  4, 10,  5,  3, 11,  7,  4,  1])

In [100]:
decode(x[0].numpy())

'94+53=741'

In [101]:
decode(y[0].numpy())

'*****741 '

Finally, the task will become conditioned generation rather than free generation, so the final piece of the code from the previous script that runs after the model is trained:

```
context = torch.zeros((1, 1), dtype=torch.long).to(device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

```

needs to be modified.



In [104]:
device = 'cpu'
context_batch = get_batch()[0].to(device)
print(context_batch.split(1, dim=0))
# for context in context_batch:
#     print(decode(m.generate(context, max_new_tokens=3)[0].tolist()))

(tensor([[ 7,  4, 10,  2,  8, 11,  2,  0,  1]]), tensor([[ 3,  5, 10,  8,  8, 11,  3,  2,  1]]), tensor([[ 2,  0, 10,  3,  5, 11,  5,  5, 12]]), tensor([[12,  9, 10,  7,  2, 11,  1,  8, 12]]), tensor([[ 2,  3, 10,  6,  3, 11,  6,  8, 12]]), tensor([[ 9,  8, 10,  4,  8, 11,  6,  4,  1]]), tensor([[ 9,  8, 10,  3,  5, 11,  3,  3,  1]]), tensor([[ 8,  1, 10,  9,  5, 11,  6,  7,  1]]), tensor([[ 2,  3, 10,  2,  2, 11,  5,  4, 12]]), tensor([[ 6,  1, 10,  9,  5, 11,  6,  5,  1]]), tensor([[ 3,  6, 10,  1,  1, 11,  7,  4, 12]]), tensor([[ 5,  4, 10,  1,  2, 11,  6,  6, 12]]), tensor([[ 2,  2, 10,  8,  8, 11,  0,  1,  1]]), tensor([[ 9,  8, 10,  2,  9, 11,  7,  2,  1]]), tensor([[ 1,  6, 10,  6,  1, 11,  7,  7, 12]]), tensor([[ 8,  3, 10,  8,  8, 11,  1,  7,  1]]), tensor([[ 8,  5, 10,  1,  2, 11,  7,  9, 12]]), tensor([[ 5,  8, 10,  1,  8, 11,  6,  7, 12]]), tensor([[ 4,  8, 10,  9,  9, 11,  7,  4,  1]]), tensor([[ 1,  1, 10,  6,  0, 11,  1,  7, 12]]), tensor([[ 1,  8, 10,  7,  5, 11,  3,  9